---

**CONEXIÓN CON DRIVE**

---

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---

**LIBRERÍAS**

---

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

---

**ANÁLISIS EXPLORATORIO DE DATOS (EDA)**

---

**Archivo: BTC-USD_Yahoo.csv**

In [18]:
# Lee el archivo y carga el dataframe

df_btc_usd_yahoo = pd.read_excel('/content/drive/MyDrive/c16-109-n-data-bi/Data en proceso Nico/PARES.xlsx', sheet_name='BTC')
df_btc_usd_yahoo.head()

,ID,Unnamed: 1,Par,Fecha,BTC Apertura,BTC Maximo,BTC Minimo,BTC Cierre,BTC Media,BTC % Media,BTC Volumen,BTC % Volumen
0,41899,41899,BTC/USD,2014-09-17,465.864014,468.174011,452.421997,457.334015,461.599015,0.000000,21056800,0.000000
1,41900,41900,BTC/USD,2014-09-18,456.859985,456.859985,413.104004,424.440002,440.649993,-0.045384,34483200,0.637628
2,41901,41901,BTC/USD,2014-09-19,424.102997,427.834991,384.532013,394.795990,409.449494,-0.112976,37919700,0.800829
3,41902,41902,BTC/USD,2014-09-20,394.673004,423.295990,389.882996,408.903992,401.788498,-0.129572,36863600,0.750674
4,41903,41903,BTC/USD,2014-09-21,408.084991,412.425995,393.181000,398.821014,403.453003,-0.125966,26580100,0.262305


In [19]:
# Información del dataframe, datos faltantes y tipo de datos
df_btc_usd_yahoo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3393 entries, 0 to 3392
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             3393 non-null   int64         
 1   Unnamed: 1     3393 non-null   int64         
 2   Par            3393 non-null   object        
 3   Fecha          3393 non-null   datetime64[ns]
 4   BTC Apertura   3393 non-null   float64       
 5   BTC Maximo     3393 non-null   float64       
 6   BTC Minimo     3393 non-null   float64       
 7   BTC Cierre     3393 non-null   float64       
 8   BTC Media      3393 non-null   float64       
 9   BTC % Media    3393 non-null   float64       
 10  BTC Volumen    3393 non-null   int64         
 11  BTC % Volumen  3393 non-null   float64       
dtypes: datetime64[ns](1), float64(7), int64(3), object(1)
memory usage: 318.2+ KB


In [20]:
df_btc_usd_yahoo.shape

(3393, 12)

In [21]:
# Porcentajes de valores nulos por columnas
porcentaje_de_nulos= df_btc_usd_yahoo.isnull().mean()*100
porcentaje_de_nulos

ID               0.0
Unnamed: 1       0.0
Par              0.0
Fecha            0.0
BTC Apertura     0.0
BTC Maximo       0.0
BTC Minimo       0.0
BTC Cierre       0.0
BTC Media        0.0
BTC % Media      0.0
BTC Volumen      0.0
BTC % Volumen    0.0
dtype: float64

In [22]:
fecha_minima= df_btc_usd_yahoo['Fecha'].min()
print (fecha_minima)

fecha_maxima= df_btc_usd_yahoo['Fecha'].max()
print (fecha_maxima)

2014-09-17 00:00:00
2023-12-31 00:00:00


---

**MACHINE LEARNING**

---

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 1. Preprocesamiento de datos
df_btc_usd_yahoo.set_index('Fecha', inplace=True)  # Establecer 'Date' como el índice

# Crear la columna 'Target' que indica si el precio aumenta o disminuye
df_btc_usd_yahoo['Target'] = (df_btc_usd_yahoo['BTC Cierre'] > df_btc_usd_yahoo['BTC Cierre'].shift(1)).astype(int)

# Seleccionar los datos del último año para test y los anteriores para entrenamiento
test_end_date = df_btc_usd_yahoo.index[-1]
train_end_date = test_end_date - pd.DateOffset(years=1)

X_train = df_btc_usd_yahoo.loc[df_btc_usd_yahoo.index <= train_end_date].drop(['Par', 'Target'], axis=1)
y_train = df_btc_usd_yahoo.loc[df_btc_usd_yahoo.index <= train_end_date]['Target']

X_test = df_btc_usd_yahoo.loc[df_btc_usd_yahoo.index > train_end_date].drop(['Par', 'Target'], axis=1)
y_test = df_btc_usd_yahoo.loc[df_btc_usd_yahoo.index > train_end_date]['Target']

# 2. Entrenamiento del modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 3. Evaluación del modelo
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))




              precision    recall  f1-score   support

           0       0.55      0.93      0.69       183
           1       0.78      0.25      0.37       182

    accuracy                           0.59       365
   macro avg       0.66      0.59      0.53       365
weighted avg       0.66      0.59      0.53       365



In [26]:
import pandas as pd

def obtener_predicciones_para_mes(mes, X_test, df_btc_usd_yahoo, model):
    # Filtrar las características para el mes dado
    caracteristicas_mes = X_test[X_test.index.month == mes]

    # Hacer las predicciones para todas las características del mes
    predicciones = []
    resultados_reales = []
    fechas = []
    for fecha, caracteristicas in caracteristicas_mes.iterrows():
        prediccion = model.predict([caracteristicas])[0]
        resultado_prediccion = 'Aumenta' if prediccion == 1 else 'Disminuye'
        predicciones.append(resultado_prediccion)

        # Obtener el resultado real
        precio_dia_actual = df_btc_usd_yahoo.loc[fecha]['BTC Cierre']
        precio_dia_anterior = df_btc_usd_yahoo.loc[fecha - pd.Timedelta(days=1)]['BTC Cierre']
        resultado_real = 'Aumenta' if precio_dia_actual > precio_dia_anterior else 'Disminuye'
        resultados_reales.append(resultado_real)
        fechas.append(fecha)

    # Crear DataFrame con las fechas, predicciones y resultados reales
    resultados_df = pd.DataFrame({
        'Fecha': fechas,
        'Predicción': predicciones,
        'Resultado Real': resultados_reales
    })


    # Calcular el porcentaje de coincidencias correctas e incorrectas
    coincidencias_correctas = sum(1 for pred, real in zip(predicciones, resultados_reales) if pred == real)
    coincidencias_totales = len(predicciones)
    porcentaje_correcto = (coincidencias_correctas / coincidencias_totales) * 100
    porcentaje_incorrecto = 100 - porcentaje_correcto

    print("Resultados:")
    print(resultados_df)
    print(f"\nPorcentaje de coincidencias correctas: {porcentaje_correcto:.2f}%")
    print(f"Porcentaje de coincidencias incorrectas: {porcentaje_incorrecto:.2f}%")

# Ejemplo de uso
mes = 9  # Por ejemplo, para julio
obtener_predicciones_para_mes(mes, X_test, df_btc_usd_yahoo, model)


Resultados:
        Fecha Predicción Resultado Real
0  2023-09-01    Aumenta      Disminuye
1  2023-09-02  Disminuye        Aumenta
2  2023-09-03  Disminuye        Aumenta
3  2023-09-04  Disminuye      Disminuye
4  2023-09-05  Disminuye      Disminuye
5  2023-09-06  Disminuye      Disminuye
6  2023-09-07  Disminuye        Aumenta
7  2023-09-08  Disminuye      Disminuye
8  2023-09-09  Disminuye      Disminuye
9  2023-09-10  Disminuye      Disminuye
10 2023-09-11  Disminuye      Disminuye
11 2023-09-12  Disminuye        Aumenta
12 2023-09-13  Disminuye        Aumenta
13 2023-09-14  Disminuye        Aumenta
14 2023-09-15  Disminuye        Aumenta
15 2023-09-16  Disminuye      Disminuye
16 2023-09-17  Disminuye      Disminuye
17 2023-09-18  Disminuye        Aumenta
18 2023-09-19  Disminuye        Aumenta
19 2023-09-20  Disminuye      Disminuye
20 2023-09-21  Disminuye      Disminuye
21 2023-09-22  Disminuye        Aumenta
22 2023-09-23  Disminuye      Disminuye
23 2023-09-24  Disminuye    